In [25]:
import pandas as pd
import numpy as np
from math import ceil
import pickle

import sys

sys.path.append("..")

from src.model.bpmf import BPMF
from src.model.pmf import PMF
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
with open(f"../data/ml-10m/train_users_dict.pkl", "rb") as handle:
    bpmf = pickle.load(handle)

In [ ]:
len(bpmf.keys()) * 0.20

In [ ]:
len(bpmf.keys())

In [26]:
df = pd.read_csv("../data/ml-20m/ratings.csv")
users_num = max(df["user_id"]) + 1
items_num = max(df["item_id"]) + 1
users_num, items_num

FileNotFoundError: [Errno 2] No such file or directory: '../data/ml-20m/ratings.csv'

In [17]:
round((1 - (len(df) / (users_num * items_num))) * 100, 2)

95.74

In [18]:
round(df.rating.mean(), 2)

3.58

In [5]:
round(df.rating.std(), 2)

1.13

In [9]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio

# Substitua 'items_num' pelo número desejado de items e 'self.n_groups' pelo número de grupos desejados
items_num = 10681  # Exemplo: 1000 items
n_groups = 10  # Exemplo: 10 grupos

# Simulando a geração de 'w' com os parâmetros fornecidos
z = np.random.geometric(p=0.35, size=items_num)
w = z % n_groups
w = [i if i > 0 else n_groups for i in w]
w.sort()  # Criando o histograma com Plotly
fig = go.Figure(data=[go.Histogram(x=w, nbinsx=n_groups)])

# Atualizando o layout do gráfico para torná-lo mais informativo
fig.update_layout(
    title="Distribuição dos Grupos de Itens",
    xaxis_title="Grupos",
    yaxis_title="Contagem de Itens",
    bargap=0.2,  # Espaço entre as barras do histograma
    width=900,
    font=dict(
        size=20,  # Set the font size here
    ),
)

fig.update_xaxes(type="category")

# Exibindo o gráfico
pio.show(fig)

In [12]:
emb_model = PMF(15400, 1000, 100).to("cuda")
emb_model.load_state_dict(
    torch.load(
        "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/model/pmf/yahoo_trained_pmf.pt",
        map_location=torch.device("cuda"),
    )
)

user_embeddings = emb_model.user_embeddings.weight.data
item_embeddings = emb_model.item_embeddings.weight.data
ratings = pd.read_csv(
    "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/data/yahoo/ratings.csv"
)
grouped = ratings.groupby("user_id")

all_user_intent_yahoo = []
for user, group in grouped:
    user_intent = pd.DataFrame(
        item_embeddings[list(group["item_id"].values)].cpu().numpy()
    )
    user_intent = cosine_similarity(user_intent, user_intent)[
        np.triu_indices(user_intent.shape[0], k=1)
    ]
    user_intent = (user_intent + 1) / 2
    user_intent = user_intent.mean()
    user_intent = 1 - user_intent
    all_user_intent_yahoo.append(user_intent)

In [13]:
emb_model = PMF(943, 1682, 100).to("cuda")
emb_model.load_state_dict(
    torch.load(
        "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/model/pmf/ml-100k_trained_pmf.pt",
        map_location=torch.device("cuda"),
    )
)

user_embeddings = emb_model.user_embeddings.weight.data
item_embeddings = emb_model.item_embeddings.weight.data
ratings = pd.read_csv(
    "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/data/ml-100k/ratings.csv"
)
grouped = ratings.groupby("user_id")

all_user_intent_100k = []
for user, group in grouped:
    user_intent = pd.DataFrame(
        item_embeddings[list(group["item_id"].values)].cpu().numpy()
    )
    user_intent = cosine_similarity(user_intent, user_intent)[
        np.triu_indices(user_intent.shape[0], k=1)
    ]
    user_intent = (user_intent + 1) / 2
    user_intent = user_intent.mean()
    user_intent = 1 - user_intent
    all_user_intent_100k.append(user_intent)

In [14]:
emb_model = PMF(6040, 3883, 100).to("cuda")
emb_model.load_state_dict(
    torch.load(
        "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/model/pmf/ml-1m_trained_pmf.pt",
        map_location=torch.device("cuda"),
    )
)

user_embeddings = emb_model.user_embeddings.weight.data
item_embeddings = emb_model.item_embeddings.weight.data
ratings = pd.read_csv(
    "/home/luana/data/CEIA/Rurax-Moblix/Recommender_system_via_deep_RL/data/ml-1m/ratings.csv"
)
grouped = ratings.groupby("user_id")

all_user_intent_1m = []
for user, group in grouped:
    user_intent = pd.DataFrame(
        item_embeddings[list(group["item_id"].values)].cpu().numpy()
    )
    user_intent = cosine_similarity(user_intent, user_intent)[
        np.triu_indices(user_intent.shape[0], k=1)
    ]
    user_intent = (user_intent + 1) / 2
    user_intent = user_intent.mean()
    user_intent = 1 - user_intent
    all_user_intent_1m.append(user_intent)

In [15]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Cria uma figura com subplots (3 linhas, 1 coluna)
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=("MovieLens (100k)", "MovieLens (1M)", "Yahoo (R3)"),
)
fig.add_trace(go.Histogram(x=all_user_intent_100k, name="100k"), row=1, col=1)

# Adiciona um histograma para all_user_intent_1m na primeira linha
fig.add_trace(go.Histogram(x=all_user_intent_1m, name="1M"), row=1, col=2)

# Adiciona um histograma para all_user_intent_10m na segunda linha
# fig.add_trace(go.Histogram(x=all_user_intent_10m, name="10m"), row=2, col=1)

# Adiciona um histograma para yahoo na terceira linha
fig.add_trace(go.Histogram(x=all_user_intent_yahoo, name="YM"), row=1, col=3)

# Atualiza o layout do subplot para torná-lo mais informativo
fig.update_layout(
    title_text="Distribuição da afinidade dos usuários",
    height=400,  # Ajuste a altura total da figura (3 subplots)
    width=1200,
    font=dict(size=16),  # Ajuste o tamanho da fonte aqui
)

# Atualizando configurações específicas de cada subplot
fig.update_xaxes(
    title_text="Afinidade dos usuários", row=3, col=1
)  # Apenas o último precisa de título no eixo X
fig.update_yaxes(
    title_text="Contagem", row=2, col=1
)  # Exemplo de adicionar título no eixo Y do subplot do meio

# Ajustando o espaço entre os subplots
fig.update_layout(bargap=0.2)  # Espaço entre as barras do histograma
fig.update_xaxes(range=[0, 1], row=1, col=1)
fig.update_xaxes(range=[0, 1], row=1, col=2)
fig.update_xaxes(range=[0, 1], row=1, col=3)
pio.show(fig)

In [23]:
x = 94.1275
round((x / 100), 3)

0.941

In [ ]:
0.958, 0.905, 21.32

In [16]:
x = 21.3196
round(x, 3)

21.32

In [ ]:
0.881 - x
0.875 - 100

In [24]:
round(((0.958 * 100) / 0.941) - 100, 1)

1.8

In [ ]:
0.955	0.877	19.656
0.906                           & 0.913                           & 9.716